In [6]:
import pandas as pd
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess

In [3]:
df = pd.read_csv('../../data/train_data.csv')
df.head(10)

,clean_tweet,cyberbullying_type
0,hate ppl high school used bully hot omg love m...,age
1,kat andre asshole omg mkr,not_cyberbullying
2,new access trading cause need high level opini...,age
3,fuck david duke racist think america belong du...,ethnicity
4,may say lot hate apologetic army hope choke ev...,other_cyberbullying
5,breanichole rapeisntokay made gay joke rape jo...,gender
6,hindustan time report gay rape joke taken obje...,gender
7,willy real man know able cook one attractive q...,gender
8,notallmen evolving notallmen evolved blameonen...,gender
9,anirban stupid mamata regime hundred bjp worke...,religion


In [9]:
df.shape

(28614, 2)

In [13]:
df.clean_tweet = df.clean_tweet.astype(str)
df.clean_tweet.dtype

dtype('O')

In [16]:
# replacing null values with empty string
# to fix decoder error
df.clean_tweet = df.clean_tweet.fillna('')

In [7]:
simple_preprocess(df.clean_tweet[0])

['hate',
 'ppl',
 'high',
 'school',
 'used',
 'bully',
 'hot',
 'omg',
 'love',
 'makeup',
 'get',
 'ring',
 'wan',
 'sometime',
 'fuck']

In [19]:
tweets = df.clean_tweet.apply(simple_preprocess)
tweets.head()

0    [hate, ppl, high, school, used, bully, hot, om...
1                      [kat, andre, asshole, omg, mkr]
2    [new, access, trading, cause, need, high, leve...
3    [fuck, david, duke, racist, think, america, be...
4    [may, say, lot, hate, apologetic, army, hope, ...
Name: clean_tweet, dtype: object

In [20]:
# spawning a Word2Vec model
model = Word2Vec(window=5, min_count=2)

# building vocabulary from entire corpus
model.build_vocab(tweets, progress_per=250)

In [26]:
model.vector_size, model.workers, model.epochs, model.corpus_count, model.corpus_total_words

(100, 3, 5, 28614, 363508)

In [71]:
# training the word2vec
model.train(tweets, total_examples=model.corpus_count, epochs=5)

(1528271, 1817540)

In [33]:
# saving the model
# model.save("../../models/word2vec.model")

In [75]:
# loading a trained model
model = Word2Vec.load('../../models/word2vec.model')

In [34]:
# top 10 similar words to bully
model.wv.most_similar('bully')

[('middle', 0.965322732925415),
 ('elementary', 0.9573146104812622),
 ('high', 0.9516770839691162),
 ('graduation', 0.9479526877403259),
 ('relentlessly', 0.9460520148277283),
 ('confrontation', 0.9452431201934814),
 ('grade', 0.9413620829582214),
 ('teased', 0.9374991059303284),
 ('teacher', 0.9353218674659729),
 ('tormentor', 0.9342045187950134)]

In [40]:
# top 10 similar words to dumb
model.wv.most_similar('dumb')

[('trayvon', 0.9649240970611572),
 ('shut', 0.9571725130081177),
 ('goshawty', 0.9547345042228699),
 ('sayin', 0.9497713446617126),
 ('ignorant', 0.9482175707817078),
 ('stupid', 0.9463467001914978),
 ('spic', 0.9458626508712769),
 ('beaner', 0.9441934823989868),
 ('nigga', 0.9439464807510376),
 ('subban', 0.9413833022117615)]

In [42]:
# word-word cosine similarity
model.wv.similarity(w1='india', w2='hindu')

0.97808623

In [56]:
# word-word cosine similarity
model.wv.similarity(w1='bully', w2='teased')

0.93749917

In [80]:
# extracting the word vectors from the model
word_vectors = (pd.DataFrame([model.wv.get_vector(str(n)) for n in model.wv.key_to_index], index = model.wv.key_to_index))

print(df_w2v.shape)
word_vectors.sample(10)

(14964, 100)


,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
nice,-0.192781,0.428249,0.163581,0.179935,0.225132,-0.952760,0.405003,1.078805,-0.556530,-0.293762,...,0.495027,0.256294,0.081433,0.399876,0.870979,0.357137,0.007828,-0.456908,0.254847,-0.234902
sheeran,-0.007434,0.017735,0.014807,0.006584,-0.006980,-0.051160,0.011977,0.061541,-0.026173,-0.011591,...,0.026947,0.002599,0.000509,0.011770,0.038903,0.019126,0.006056,-0.015366,-0.003382,-0.010420
discriminated,0.018713,-0.001984,-0.018869,-0.036782,-0.010157,-0.091945,0.013388,0.086454,-0.028530,-0.025232,...,0.051572,0.033785,-0.001509,0.012612,0.063108,0.020124,0.013529,-0.036124,0.023897,-0.022599
typeonegative,0.003733,-0.011045,-0.014000,-0.012526,-0.000878,-0.005549,0.009506,0.036990,-0.008973,0.008513,...,0.022193,0.005509,-0.007446,0.009933,0.006811,0.012697,-0.017803,-0.009518,0.023444,-0.012139
objection,-0.001627,-0.002641,-0.009385,-0.012354,-0.015650,-0.016477,-0.000673,0.026697,0.014142,0.001016,...,-0.002933,0.014575,-0.001871,-0.003120,0.000940,0.004874,-0.007170,-0.018893,0.003600,-0.008676
professora,-0.022106,0.023572,0.024930,0.030998,-0.004385,-0.076845,0.023927,0.100673,-0.032465,-0.018333,...,0.010405,0.015398,-0.006973,0.032991,0.060261,0.034890,0.007864,-0.051127,0.015242,-0.035894
slasher,-0.006892,0.013456,0.016699,0.001025,-0.010877,-0.035924,0.001811,0.038645,-0.014912,-0.003330,...,0.008048,0.001483,-0.004797,0.004991,0.030776,0.014187,0.008514,-0.015796,0.005368,-0.021430
higher,-0.045284,0.134403,0.067103,0.064914,0.000813,-0.461645,0.078339,0.509531,-0.162674,-0.164502,...,0.196889,0.143719,0.009727,0.136243,0.330119,0.117404,0.006872,-0.227228,0.107134,-0.168955
nessa,-0.001683,0.006513,0.008208,0.000590,-0.001875,-0.048592,0.015787,0.042553,-0.001710,-0.012779,...,0.015412,0.014694,0.012261,0.005298,0.023851,0.006722,0.014951,-0.017630,-0.013593,-0.010662
similar,-0.029353,0.104890,0.046148,0.029808,-0.020664,-0.362088,0.030800,0.385523,-0.101135,-0.129910,...,0.132651,0.121552,-0.011716,0.094555,0.242424,0.091967,0.019743,-0.174501,0.106097,-0.151711
